In [ ]:
import os
import json
import requests
from openai import AzureOpenAI
from IPython.display import Image, display

# --- Load Azure credentials from cred.json ---
with open("cred.json", "r") as f:
    config = json.load(f)

api_key = config["AZURE_INFERENCE_KEY"]
endpoint = config["AZURE_INFERENCE_ENDPOINT"]

# --- Initialize Azure OpenAI client ---
client = AzureOpenAI(
    api_key=api_key,
    azure_endpoint=endpoint,
    api_version="2024-02-01"   # image API version
)

# --- Prompt for image generation ---
prompt = "A watercolor painting of fresh fruits and vegetables arranged in a heart shape"

result = client.images.generate(
    model="dall-e-3",
    prompt=prompt,
    n=1,
    size="1024x1024"
)

# --- Get the image URL ---
image_url = result.data[0].url
print("🔗 Generated Image URL:", image_url)

# --- Download image ---
os.makedirs("generated_images", exist_ok=True)
filename = "generated_images/fruits_heart.png"

response = requests.get(image_url)
response.raise_for_status()

with open(filename, "wb") as f:
    f.write(response.content)

# --- Display image directly in Jupyter ---
display(Image(filename=filename))


In [ ]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import ImageEmbeddingsClient
from azure.ai.inference.models import ImageEmbeddingInput

# --- Config ---
endpoint = "your-end-point"
model_name = "Cohere-embed-v3-english"   # 👈 Use your deployed image embedding model name
api_key = "your-end-point-key"  # or replace with your key string

# --- Client ---
client = ImageEmbeddingsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(api_key)
)

# --- Input image ---
img_input = ImageEmbeddingInput.load(
    image_file="chart.png",
    image_format="png"
)

# --- Generate embeddings ---
response = client.embed(
    input=[img_input],
    model=model_name
)

# --- Inspect results ---
for item in response.data:
    length = len(item.embedding)
    print(
        f"image[{item.index}]: length={length}, "
        f"[{item.embedding[0]:.4f}, {item.embedding[1]:.4f}, "
        f"..., {item.embedding[length-2]:.4f}, {item.embedding[length-1]:.4f}]"
    )

print(response.usage)
